In [27]:
pwd

'/Users/mvferr/work/dev/fund'

In [2]:
# using Pandas from my virtualenv
import os
import sys
sys.path.append( os.environ['VIRTUAL_ENV'] + "/lib/python3.9/site-packages" )
sys.path

['/Users/mvferr/work/dev/fund',
 '/usr/local/Cellar/python@3.9/3.9.1_1/Frameworks/Python.framework/Versions/3.9/lib/python39.zip',
 '/usr/local/Cellar/python@3.9/3.9.1_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9',
 '/usr/local/Cellar/python@3.9/3.9.1_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '',
 '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages',
 '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages/IPython/extensions',
 '/Users/mvferr/.ipython',
 '../venv/lib/python3.9/site-packages',
 '/Users/mvferr/work/dev/fund/venv/lib/python3.9/site-packages']

In [3]:
import requests

url = 'http://www.fundamentus.com.br/resultado.php'
hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
       'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01',
       'Accept-Encoding': 'gzip, deflate',
       }
params = {'pl_min': '0.01'}

content = requests.post(url, headers=hdr, data=params)

In [24]:
import pandas as pd

## parse
df = pd.read_html(content.text, decimal=",", thousands='.')[0]

In [25]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,MLPA4,45.00,0.01,0.00,0.001,"0,00%",0.003,0.00,0.01,0.00,...,-2.06,"6,66%","8,92%",0.00,"0,00%","45,13%",0.0,4.457800e+07,0.00,"-7,51%"
1,CAFE3,2.00,0.02,-0.01,0.283,"0,00%",0.015,-0.01,-0.53,-0.01,...,-53.96,"-53,07%","2.163,80%",0.02,"-2,81%","-64,17%",0.0,-3.773870e+08,-0.98,"-11,30%"
2,CORR3,12.00,0.03,0.05,0.016,"0,00%",0.012,0.02,0.03,0.16,...,-0.17,"58,25%","53,90%",13.09,"48,27%","164,97%",0.0,5.563000e+06,0.00,"37,05%"
3,BSCT3,4.05,0.04,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","9,05%",0.0,2.129600e+08,0.00,"1,92%"
4,RCTB42,20.00,0.05,0.00,0.005,"0,00%",0.000,0.00,-0.01,-0.00,...,8.82,"-104,09%","11,00%",5.30,"-8,18%","1,79%",0.0,1.591550e+09,0.17,"42,35%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,GUAR4,151.00,1661.32,15.52,11.645,"0,00%",5.746,30.19,-766.71,-186.18,...,189.56,"-1,52%","0,70%",1.47,"-1,06%","0,93%",0.0,4.855730e+09,0.93,"4,03%"
517,APER3,31.20,3511.53,2.00,3.555,"0,17%",1.216,9.88,-1072.38,-9.70,...,47.58,"-0,33%","0,10%",1.86,"-0,14%","0,06%",1020610.0,1.790790e+08,0.00,"-8,08%"
518,BIDI11,99.78,19174.50,7.82,0.000,"0,21%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,04%",76915600.0,3.251960e+09,0.00,"17,23%"
519,BIDI4,33.27,19180.30,7.82,0.000,"0,21%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,04%",53792200.0,3.251960e+09,0.00,"17,23%"


In [72]:
df.name = 'Fundamentus'

# update index
df.index = df['Papel']
df.index.name = 'Papel'

# rename columns
df3 = pd.DataFrame()
df3['papel'] = df['Papel']
df3['pl']    = df['P/L']
df3['pvp']   = df['P/VP']
df3['patrliq'] = df['Patrim. Líq']


df3
print(df3.to_csv(index=False, header=True, sep=';', doublequote='"', decimal='.', float_format='%15.2f' ))

papel;pl;pvp;patrliq
MLPA4;           0.01;           0.00;    44578000.00
CAFE3;           0.02;          -0.01;  -377387000.00
CORR3;           0.03;           0.05;     5563000.00
BSCT3;           0.04;           0.00;   212960000.00
RCTB42;           0.05;           0.00;  1591550000.00
BPHA3;           0.06;          -0.24;  -292294000.00
LREN4;           0.07;           0.02;  5287760000.00
RCTB41;           0.08;           0.00;  1591550000.00
BRSR4;           0.08;           0.01;  8352800000.00
RCTB31;           0.10;           0.00;  1591550000.00
MTIG3;           0.11;           0.05;    27819200.00
CREM4;           0.11;           0.01;   189701000.00
PALF11;           0.11;           0.06;  1840190000.00
TPRC6;           0.13;           0.01; 22847800000.00
RCTB33;           0.14;           0.00;  1591550000.00
TPRC3;           0.14;           0.01; 22847800000.00
CORR4;           0.17;           0.29;     5563000.00
PALF5;           0.21;           0.11;  1840190000.00
PA

In [42]:
df3

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
Papel,,,,,,,,,,,,,,,,,,,,,
MLPA4,MLPA4,45.00,0.01,0.00,0.001,"0,00%",0.003,0.00,0.01,0.00,...,-2.06,"6,66%","8,92%",0.00,"0,00%","45,13%",0.0,4.457800e+07,0.00,"-7,51%"
CAFE3,CAFE3,2.00,0.02,-0.01,0.283,"0,00%",0.015,-0.01,-0.53,-0.01,...,-53.96,"-53,07%","2.163,80%",0.02,"-2,81%","-64,17%",0.0,-3.773870e+08,-0.98,"-11,30%"
CORR3,CORR3,12.00,0.03,0.05,0.016,"0,00%",0.012,0.02,0.03,0.16,...,-0.17,"58,25%","53,90%",13.09,"48,27%","164,97%",0.0,5.563000e+06,0.00,"37,05%"
BSCT3,BSCT3,4.05,0.04,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","9,05%",0.0,2.129600e+08,0.00,"1,92%"
RCTB42,RCTB42,20.00,0.05,0.00,0.005,"0,00%",0.000,0.00,-0.01,-0.00,...,8.82,"-104,09%","11,00%",5.30,"-8,18%","1,79%",0.0,1.591550e+09,0.17,"42,35%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GUAR4,GUAR4,151.00,1661.32,15.52,11.645,"0,00%",5.746,30.19,-766.71,-186.18,...,189.56,"-1,52%","0,70%",1.47,"-1,06%","0,93%",0.0,4.855730e+09,0.93,"4,03%"
APER3,APER3,31.20,3511.53,2.00,3.555,"0,17%",1.216,9.88,-1072.38,-9.70,...,47.58,"-0,33%","0,10%",1.86,"-0,14%","0,06%",1020610.0,1.790790e+08,0.00,"-8,08%"
BIDI11,BIDI11,99.78,19174.50,7.82,0.000,"0,21%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,04%",76915600.0,3.251960e+09,0.00,"17,23%"


In [ ]:
df

df[ df['P/L'] > 0 ]
df[ df['P/L'] > 0 &   df['ROE'] > 0 ]
df[ df['P/L'] > 0 &&  df['ROE'] > 0 ]
df[ df['P/L'] > 0 and df['ROE'] > 0 ]

df[ df['P/L'] > 0  ][ ['Papel','EV/EBITDA','ROE','ROIC'] ]

df.name = 'Fundamentus'
df.index.name = 'Papel'

# rename index
df.index = df['Papel']
# rename columns
df.rename( columns={'Cotação': 'cotacao', 
                    'P/L': 'pl'
                   } )

df2 = pd.DataFrame( df               , columns=['Cotação', 'P/L', 'ROE', 'ROIC', 'EV/EBIT','EV/EBITDA'] )
df2 = pd.DataFrame( df[ df['P/L']>0 ], columns=['Cotação', 'P/L', 'ROE', 'ROIC', 'EV/EBIT','EV/EBITDA'] )

df2 = df.rename(columns={
            'Papel'            : 'papel'    ,
            'Cotação'          : 'cotacao'  ,
            'P/L'              : 'pl'       ,
            'P/VP'             : 'pvp'      ,
            'PSR'              : 'psr'      ,
            'Div.Yield'        : 'dy'       ,
            'P/Ativo'          : 'pa'       ,
            'P/Cap.Giro'       : 'pcg'      ,
            'P/EBIT'           : 'pebit'    ,
            'P/Ativ Circ.Liq'  : 'pacl'     ,
            'EV/EBIT'          : 'evebit'   ,
            'EV/EBITDA'        : 'evebitda' ,
            'Mrg Ebit'         : 'mrgebit'  ,
            'Mrg. Líq.'        : 'mrgliq'   ,
            'ROIC'             : 'roic'     ,
            'ROE'              : 'roe'      ,
            'Liq. Corr.'       : 'liqc'     ,
            'Liq.2meses'       : 'liq2m'    ,
            'Patrim. Líq'      : 'patrliq'  ,
            'Dív.Brut/ Patrim.': 'divbpatr' ,
            'Cresc. Rec.5a'    : 'c5y'      ,
            })
